In [ ]:
using Distributed
ifelse(nprocs()<2, addprocs(3), nothing);

In [ ]:
using Plots 
# using Revise 
using LatticeQM 

# Tutorial: Twisted honeycomb lattices

In [ ]:
@time lat = Geometries.honeycomb_twisted(8)

In [ ]:
plot(lat, 3; supercell=Structure.Lattices.getneighborcells(lat, 1; halfspace=false, innerpoints=true, excludeorigin=false), sort="layer", markersize=3)

## Model

In [ ]:
@time valley = Operators.valley(lat; spinhalf=false) 
@time H = Operators.graphene(lat; tz=0.45, format=:sparse, mode=:nospin) # specifying format is important here!
Operators.setfilling!(H, 0.5; nk=12) 
nothing 

In [ ]:
@time valley = Operators.valley(lat; spinhalf=false) 
@time H = Operators.graphene(lat; tz=0.45, format=:sparse, mode=:nospin) # specifying format is important here!
Operators.setfilling!(H, 0.5; nk=12) 
nothing 

### Bands

In [ ]:
ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=100)
bands = getbands(H, ks, valley; format=:sparse, num_bands=25) 

In [ ]:
p1 = plot(bands; size=(300,250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)
p2 = plot(bands; ylim=(-0.0025,0.0025), size=(300,250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

plot(p1,p2; size=(600,200))

In [ ]:
H = Operators.graphene(lat; mode=:spinhalf, format=:sparse, tz=0.45)
Operators.addrashba!(H, lat, r -> 0.04 * sign(r[3]))
Operators.addzeeman!(H, lat, r -> 0.04 * [0.0, 0.0, sign(r[3])])
Operators.setfilling!(H, 0.5; nk=12)

# kgrid = LatticeQM.Utils.randomgrid(;nk=8^2, rot_symmetry=6, B=Structure.getB(lat))
kgrid = Structure.regulargrid(;nk=8^2)
ωs = collect(range(-0.0015, length=100, stop=0.001))
ldos = Spectrum.ldos(H, kgrid, ωs; Γ=0.0005, format=:sparse, num_bands=25);

In [ ]:
ldos0 = ldos[1:2:hopdim(H)] + ldos[2:2:hopdim(H)]
plot(lat, ldos0; clims=(0,maximum(ldos)), markercolor=:inferno, colorbar=true)

In [ ]:
ldos0 = ldos[1:2:hopdim(H)] - ldos[2:2:hopdim(H)]
cmax = maximum(abs.(ldos0))
plot(lat, ldos0; clims=(-cmax,cmax), colorbar=true)